In [1]:
from threading import Thread
from math import pi

import rclpy
from rclpy.callback_groups import ReentrantCallbackGroup
from rclpy.node import Node

from pymoveit2 import MoveIt2, MoveIt2State
from pymoveit2.robots import ur
from ur_msgs.srv import SetIO


rclpy.init()

# Create node for this example
node = Node("TEST")
callback_group = ReentrantCallbackGroup()


moveit2 = MoveIt2(
    node=node,
    joint_names=ur.joint_names(),
    base_link_name=ur.base_link_name(),
    end_effector_name=ur.end_effector_name(),
    group_name=ur.MOVE_GROUP_ARM,
    callback_group=callback_group,
)

# Spin the node in background thread(s) and wait a bit for initialization
executor = rclpy.executors.MultiThreadedExecutor(2)
executor.add_node(node)
executor_thread = Thread(target=executor.spin, daemon=True, args=())
executor_thread.start()
node.create_rate(1.0).sleep()

# Scale down velocity and acceleration of joints (percentage of maximum)
moveit2.max_velocity = 0.1
moveit2.max_acceleration = 0.1
synchronous = True
cancel_after_secs = 0.0
cartesian = False
cartesian_max_step = 0.0025
cartesian_fraction_threshold = 0.0
cartesian_jump_threshold = 0.0
cartesian_avoid_collisions = False
moveit2.planner_id = "PTP"

In [8]:
home_position = [0.0, -pi / 2, pi / 2, 0.0, 0.0, 0.0]
node.get_logger().info("Moving to home position")
traj = moveit2.move_to_configuration(home_position)
if traj is None:
    node.get_logger().error("Failed to move to home position")

[INFO] [1731598805.042221144] [TEST]: Moving to home position


In [9]:
moveit2.execute(traj)
success = moveit2.wait_until_executed()
if not success:
    node.get_logger().error("Failed to execute trajectory")

In [6]:
position = [0.5, 0, 0.5]
quat_xyzw = [1.0, 0.0, 0.0, 0.0]


node.get_logger().info(f"Moving to {{position: {list(position)}, quat_xyzw: {list(quat_xyzw)}}}")
traj = moveit2.move_to_pose(
    position=position,
    quat_xyzw=quat_xyzw,
    cartesian=cartesian,
    cartesian_max_step=cartesian_max_step,
    cartesian_fraction_threshold=cartesian_fraction_threshold,
)

[INFO] [1731598436.322608528] [TEST]: Moving to {position: [0.5, 0, 0.5], quat_xyzw: [1.0, 0.0, 0.0, 0.0]}


In [7]:
moveit2.execute(traj)
success = moveit2.wait_until_executed()
if not success:
    node.get_logger().error("Failed to move to pose")